# Лабораторные работы 1-5 (Медведев К.В.)

# ЛР 1 (KNN)

## 1.1 Выбор начальных условий

Датасеты для задач классификации и регрессии, метрики моделей, а также обоснования их выбора приведены в `README.md` файле репозитория.